In [ ]:
#fn = '/Users/savoie/projects/monthly_sea_ice_extent_tools/source/data/csvify/Dec/N_12_area.txt'
fn = '/Users/savoie/projects/monthly_sea_ice_extent_tools/source/data/csvify/Jun/N_06_area.txt'
#fn = '/Users/savoie/projects/monthly_sea_ice_extent_tools/source/data/csvify/Sep/N_09_area.txt'

In [ ]:
%matplotlib inline
import matplotlib.pylab
import pandas as pd
import numpy as np
import matplotlib as mpl
import datetime as dt
from scipy.stats import linregress
pd.options.display.mpl_style = 'default'


In [ ]:
def slurp_csv(filename):
    data = pd.read_csv(filename,
                       error_bad_lines=False,
                       skipinitialspace=True,
                       delimiter='\s+' )
    data.rename(columns={'mo': 'month', "region":"hemisphere"}, inplace=True)
    return data.dropna()


In [ ]:
data = slurp_csv(fn)

Set missing data to None Type.

In [ ]:
data.loc[ data.extent <= 0, ['extent', 'area', 'data_type']] = None
data = data.convert_objects(convert_numeric=True)

In [ ]:
data.dtypes

In [ ]:
a = data.copy()

Create an column of Date Periods and set it as the index.

In [ ]:
a['dates']= [pd.Period(dt.date(int(x[0]), int(x[1]), 1), "M") for x in zip(a.values[:,0], a.values[:,1])]
a = a.set_index('dates')

In [ ]:
a.head()

create new dataframe for the (rank sorted stuff)

In [ ]:
b = pd.DataFrame(index=a.index)

In [ ]:
#filler
b[' reordered => '] = " "

b['rank'] = pd.DataFrame(data=a['extent'].rank(ascending=1))#.rename(columns={'extent':'rank'})

In [ ]:

b['ranked_year'] = a['year']
b['ranked_extent'] = a['extent']

In [ ]:
b.sort('rank', ascending=True, inplace=True)

compute extent anomaly

In [ ]:
climatological_mean = a[(a.year >= 1981) & (a.year <= 2010)].extent.mean()

In [ ]:
a['extent_anomaly'] = a.extent - climatological_mean

In [ ]:
a.head()

I can't get expanding apply to work with two values, and I can't cram them together so I'm going to have to loop over the indices.

In [ ]:
a['trend_through_year_km2_per_year'] = None
a['p_value'] = None
a['r_value'] = None
a['stderr'] = None
a['Significant'] = False
a['% trend_through_year'] = None

In [ ]:
a[['trend_through_year_km2_per_year','p_value', 'r_value', 'stderr', '% trend_through_year']] = a[['trend_through_year_km2_per_year','p_value', 'r_value', 'stderr', '% trend_through_year']].astype(np.float64)

Article on operational definition of Statistically Meaningful Trend (may or may not be correct in our case)
http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3084280/

If one or several regressions concerning time and values in a time series, or
time and mean values from intervals into which the series has been divided,
yields r2≥0.65 and p≤0.05, then the time series is statistically meaningful.

In [ ]:
for i in range(0,len(a)):
    cum_df = a.iloc[:i+1]
    (slope, intercept, r_value, p_value, stderr) = linregress(cum_df['year'], cum_df['extent'])
    a.ix[i, ['trend_through_year_km2_per_year', 'r_value', 'p_value', 'stderr']] = round(slope, 4) * 1000000, r_value, p_value, stderr
    a.ix[i, '% trend_through_year'] =  slope / climatological_mean * 10 * 100.
    a.ix[i, 'Significant'] = (np.square(r_value) >= .65) & (p_value <= .05)



In [ ]:
# cram the two parts together 
a = a.reset_index().join(b.reset_index(drop=True))

In [ ]:
with open('../output/test.csv', 'w') as fp:
    fp.write('This is line1\n')
    fp.write('This is line2\n')
    a.to_csv(fp, header=True, float_format="%3.2f", index=False )